### Import Libraries and Set the intial variables

In [1]:
# !pip install transformers

# # подключить google drive
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [2]:
from transformers import BertModel, BertTokenizer
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

#constants
PATH = r''
MAX_LEN = 128
CLASS_NAMES = ["normal", "toxic"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SM = torch.nn.Sigmoid()

MODEL_FILE_NAME = r'model_dp_512.bin'
MODEL_NAME = 'DeepPavlov/rubert-base-cased-conversational'
THRESHOLD = 0.86

In [ ]:
class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter. Wraps an existing OPT linear layer"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module
        self.adapter = nn.Sequential(
            nn.Linear(module.in_features, rank, bias=False),
            nn.Linear(rank, module.out_features, bias=False)
        )
        nn.init.kaiming_uniform_(self.adapter[0].weight, a=5 ** 0.5)
        nn.init.zeros_(self.adapter[1].weight)

        self.adapter.to(module.weight.device)

    def forward(self, input):
        # Apply self.module and LoRA adapter, return the sum (base module outputs + adapter outputs)
        return self.module(input) + self.adapter(input)

In [3]:
class SentimentClassifier(nn.Module):

    def __init__(self, model_name_):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name_)
        
        for name, module in self.bert.named_modules():
            if 'BertSelfAttention' in repr(type(module)):
                module.query = LoRALayer(module.query, rank=512)
                module.key = LoRALayer(module.key, rank=512)
                module.value = LoRALayer(module.value, rank=512)
        
        self.out = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 600),
            nn.ReLU(),
            nn.BatchNorm1d(600),
            nn.Linear(600, 600),
            nn.ReLU(),
            nn.BatchNorm1d(600),
            nn.Linear(600, 300),
            nn.ReLU(),
            nn.BatchNorm1d(300),
            nn.Linear(300, 1),
            )

    def forward(self, input_ids, attention_mask):
        _, output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask,
          return_dict=False
        )
        return self.out(output)

In [4]:
def model_prediction(model, tokenizer, text_for_prediction):
  encoded_review = tokenizer.encode_plus(
      text_for_prediction,
      max_length=MAX_LEN,
      add_special_tokens=True,
      return_token_type_ids=False,
      padding='max_length',
      return_attention_mask=True,
      truncation=True,
      return_tensors='pt',
  )

  input_ids = encoded_review['input_ids'].to(DEVICE)
  attention_mask = encoded_review['attention_mask'].to(DEVICE)

  model.eval()
  with torch.no_grad():
    output = model(input_ids, attention_mask)

    output_probs = SM(output.flatten())
    prediction = (output_probs.flatten() > THRESHOLD).int()

  return CLASS_NAMES[prediction]

In [5]:
model = SentimentClassifier(model_name_ = MODEL_NAME)
model.load_state_dict(torch.load(PATH + MODEL_FILE_NAME))
model = model.to(DEVICE)

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model_prediction(model, tokenizer, "ты баран")

'toxic'

In [7]:
model_prediction(model, tokenizer, "пацан красавчик!")

'normal'